# Results

## Library Import

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import seaborn as sns
from matplotlib import colors as mcolors

## Helper Functions

In [ ]:
def plot_merfish_zhuang_large(adata, color_key, figsize=(6, 6), size=5, title=None):
    fig, ax = plt.subplots(figsize=figsize)
    domain_label = color_key

    unique_classes = adata.obs[domain_label].unique()
    num_classes = len(unique_classes)
    palette = sns.color_palette("tab20", num_classes)
    adata.uns[f"{domain_label}_colors"] = [mcolors.rgb2hex(c) for c in palette]

    sc.pl.embedding(
        adata, basis="spatial", color=domain_label, size=size, ax=ax, show=False, legend_loc=None
    )

    ax.set_ylim(11, 0)
    ax.set_xlim(0, 11)
    ax.axis("equal")
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(title)

    plt.show()

## Domain123 Baseline

In [ ]:
test_results = pd.read_csv(
    "../data/domain/results/domain123/2025-04-16_08-55-57/csv/version_0/test_results.csv"
)
test_results = test_results.map(
    lambda x: float(x.split("(")[-1].rstrip(")")) if isinstance(x, str) and "tensor" in x else x
)
test_results

In [ ]:
test_results_melted = test_results.melt(
    id_vars="sample_name",
    value_vars=["nmi", "homogeneity", "completeness"],
    var_name="Metric",
    value_name="Score",
)

sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(4, 2), sharex=True)

metrics = ["nmi", "homogeneity", "completeness"]
titles = ["NMI", "HOM", "COM"]

for ax, metric, title in zip(axes, metrics, titles):
    sns.boxplot(
        x="Score",
        y="Metric",
        data=test_results_melted[test_results_melted["Metric"] == metric],
        ax=ax,
        orient="h",
        dodge=False,
    )
    sns.stripplot(
        x="Score",
        y="Metric",
        data=test_results_melted[test_results_melted["Metric"] == metric],
        ax=ax,
        color="black",
        size=5,
        jitter=True,
        orient="h",
    )
    ax.set_xlim(0.2, 0.8)
    ax.set_title(title, fontsize=12)
    ax.set_xlabel("Score")
    ax.set_ylabel("")
    ax.set_yticklabels([])

    ax.set_xticks([0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85])
    ax.set_xticklabels(["", "", "0.25", "", "", "", "", "0.75", ""])

plt.tight_layout()
plt.show()

In [ ]:
merfish_small5 = sc.read_h5ad(
    "../data/domain/results/domain123/2025-04-16_08-55-57/adata_files/MERFISH_small5.h5ad"
)
merfish_small5

In [ ]:
merfish_small5.obsm["spatial"] *= -1
merfish_small5.obsm["spatial"][:, 0] *= -1

fig, ax = plt.subplots(figsize=(5, 6))
sc.pl.embedding(
    merfish_small5,
    basis="spatial",
    color="domain_annotation",
    size=60,
    title="Ground Truth",
    ax=ax,
    show=False,
)
plt.show()

fig, ax = plt.subplots(figsize=(5, 6))
sc.pl.embedding(
    merfish_small5,
    basis="spatial",
    color="leiden",
    size=60,
    title="NMI: 0.68, HOM: 0.67, COM: 0.69",
    ax=ax,
    show=False,
)
plt.show()

In [ ]:
merfish_small5.uns["leiden_colors"] = [
    merfish_small5.uns["leiden_colors"][i] for i in [1, 0, 3, 4, 2, 7, 5, 6]
]

## Domain4 on Domain123 Baseline

In [ ]:
test_results = pd.read_csv(
    "../data/domain/results/domain4/2025-04-18_11-34-38/csv/version_0/test_results.csv"
)
test_results = test_results.map(
    lambda x: float(x.split("(")[-1].rstrip(")")) if isinstance(x, str) and "tensor" in x else x
)
test_results

In [ ]:
test_results_melted = test_results.melt(
    id_vars="sample_name",
    value_vars=["nmi", "homogeneity", "completeness"],
    var_name="Metric",
    value_name="Score",
)

sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(4, 2), sharex=False)

metrics = ["nmi", "homogeneity", "completeness"]
titles = ["NMI", "HOM", "COM"]
xlims_list = [[0.58, 0.70], [0.49, 0.67], [0.59, 0.74]]
xticks_list = [
    [0.59, 0.60, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69],
    [0.51, 0.53, 0.55, 0.57, 0.59, 0.61, 0.63, 0.65],
    [0.60, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.70, 0.71, 0.72, 0.73],
]
xtick_labels_list = [
    ["", "0.60", "", "", "", "", "0.65", "", "", "", ""],
    ["", "0.53", "", "", "", "", "", "0.65"],
    ["0.60", "", "", "", "", "", "", "", "", "", "", "", "0.73", ""],
]

for ax, metric, title, xlims, xticks, xtick_labels in zip(
    axes, metrics, titles, xlims_list, xticks_list, xtick_labels_list
):
    sns.boxplot(
        x="Score",
        y="Metric",
        data=test_results_melted[test_results_melted["Metric"] == metric],
        ax=ax,
        orient="h",
        dodge=False,
    )
    sns.stripplot(
        x="Score",
        y="Metric",
        data=test_results_melted[test_results_melted["Metric"] == metric],
        ax=ax,
        color="black",
        size=5,
        jitter=True,
        orient="h",
    )
    ax.set_title(title, fontsize=12)
    ax.set_xlabel("Score")
    ax.set_ylabel("")
    ax.set_yticklabels([])

    ax.set_xlim(xlims)
    ax.set_xticks(xticks)
    ax.set_xticklabels(xtick_labels)

plt.tight_layout()
plt.show()

In [ ]:
xenium2 = sc.read_h5ad(
    "../data/domain/results/domain4/2025-04-18_11-34-38/adata_files/Xenium2.h5ad"
)
xenium2

In [ ]:
domain_label = "domain_annotation"
title = "Xenium Ground Truth"

x_midpoint = xenium2.obsm["spatial"][:, 0].mean()
left_half_xenium = xenium2[xenium2.obsm["spatial"][:, 0] < x_midpoint]

unique_classes = left_half_xenium.obs[domain_label].unique()
num_classes = len(unique_classes)

palette = sns.color_palette("tab20", num_classes)
left_half_xenium.uns[f"{domain_label}_colors"] = [mcolors.rgb2hex(c) for c in palette]

fig, ax = plt.subplots(figsize=(4, 5))
sc.pl.embedding(
    left_half_xenium,
    basis="spatial",
    color=domain_label,
    size=5,
    title=title,
    legend_loc=None,
    show=False,
    ax=ax,
)
plt.show()

In [ ]:
domain_label = "leiden"
title = "NMI: 0.60, HOM: 0.57, COM: 0.62"

x_midpoint = xenium2.obsm["spatial"][:, 0].mean()
left_half_xenium = xenium2[xenium2.obsm["spatial"][:, 0] < x_midpoint]

unique_classes = left_half_xenium.obs[domain_label].unique()
num_classes = len(unique_classes)

palette = sns.color_palette("tab20", num_classes)
left_half_xenium.uns[f"{domain_label}_colors"] = [mcolors.rgb2hex(c) for c in palette]

fig, ax = plt.subplots(figsize=(4, 5))
sc.pl.embedding(
    left_half_xenium,
    basis="spatial",
    color=domain_label,
    size=5,
    title=title,
    legend_loc=None,
    show=False,
    ax=ax,
)
plt.show()

## Domain7 Baseline

In [ ]:
test_results = pd.read_csv(
    "../data/domain/results/domain7/2025-04-22_08-31-58/csv/version_0/test_results.csv"
)
test_results = test_results.map(
    lambda x: float(x.split("(")[-1].rstrip(")")) if isinstance(x, str) and "tensor" in x else x
)
test_results

In [ ]:
def assign_dataset(sample_name):
    if "ABCA-1" in sample_name:
        return "dataset7.1"
    elif "ABCA-2" in sample_name:
        return "dataset7.2"
    elif "ABCA-3" in sample_name:
        return "dataset7.3"
    elif "ABCA-4" in sample_name:
        return "dataset7.4"
    else:
        return "Unknown"


test_results["dataset"] = test_results["sample_name"].apply(assign_dataset)
test_results

In [ ]:
test_results_melted = test_results.melt(
    id_vars="dataset",
    value_vars=["nmi", "homogeneity", "completeness"],
    var_name="Metric",
    value_name="Score",
)
data_set_nmi_mean = pd.DataFrame(test_results.groupby("dataset")["nmi"].mean())
category_order = ["dataset7.1", "dataset7.2", "dataset7.3", "dataset7.4"]
hue_order = ["dataset7.1", "dataset7.2", "dataset7.3", "dataset7.4"]
palette = sns.color_palette("tab10")[0:4]

sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(2, 5), sharex=True)

sns.barplot(
    data=data_set_nmi_mean,
    x="dataset",
    y="nmi",
    ax=axes[0],
    palette=palette,
    order=category_order,
    hue="dataset",
    hue_order=hue_order,
    legend=False,
)

axes[0].set_title("Baseline Performance", fontsize=12)
axes[0].set_xlabel("Dataset")
axes[0].set_ylabel("NMI")
axes[0].set_yticks(
    [
        0.00,
        0.04,
        0.08,
        0.12,
        0.16,
        0.20,
        0.24,
        0.28,
        0.32,
        0.36,
        0.40,
        0.44,
        0.48,
        0.52,
        0.56,
        0.60,
        0.64,
        0.68,
    ]
)
axes[0].set_yticklabels(
    ["0.0", "", "", "", "", "0.2", "", "", "", "", "0.4", "", "", "", "", "0.6", "", ""]
)

sns.boxplot(
    x="dataset",
    y="Score",
    data=test_results_melted[test_results_melted["Metric"] == "nmi"],
    ax=axes[1],
    palette=palette,
    order=category_order,
    hue="dataset",
    hue_order=hue_order,
    legend=False,
)
sns.stripplot(
    x="dataset",
    y="Score",
    data=test_results_melted[test_results_melted["Metric"] == "nmi"],
    ax=axes[1],
    color="black",
    size=5,
    jitter=True,
)

axes[1].set_xlabel("Dataset")
axes[1].set_ylabel("NMI")
axes[1].set_yticks(
    [
        0.20,
        0.22,
        0.24,
        0.26,
        0.28,
        0.30,
        0.32,
        0.34,
        0.36,
        0.38,
        0.40,
        0.42,
        0.44,
        0.46,
        0.48,
        0.50,
        0.52,
        0.54,
        0.56,
        0.58,
        0.60,
        0.62,
        0.64,
        0.66,
        0.68,
        0.70,
        0.72,
    ]
)
axes[1].set_yticklabels(
    [
        "0.2",
        "",
        "",
        "",
        "",
        "0.3",
        "",
        "",
        "",
        "",
        "0.4",
        "",
        "",
        "",
        "",
        "0.5",
        "",
        "",
        "",
        "",
        "0.6",
        "",
        "",
        "",
        "",
        "0.7",
        "",
    ]
)
axes[1].tick_params(axis="x", rotation=90, bottom=True)

plt.tight_layout()
plt.show()

In [ ]:
abca4_003 = sc.read_h5ad(
    "../data/domain/results/domain7/2025-04-22_08-31-58/adata_files/Zhuang-ABCA-4.003.h5ad"
)
abca4_003

In [ ]:
plot_merfish_zhuang_large(
    abca4_003, "domain_annotation", title="Zhuang-ABCA-4.003 Ground Truth", figsize=(10, 6)
)
plot_merfish_zhuang_large(
    abca4_003, "leiden", title="Zhuang-ABCA-4.003 NMI: 0.57, HOM: 0.52, COM: 0.62", figsize=(10, 6)
)

In [ ]:
abca2_036 = sc.read_h5ad(
    "../data/domain/results/domain7/2025-04-22_08-31-58/adata_files/Zhuang-ABCA-2.036.h5ad"
)
abca2_036

In [ ]:
plot_merfish_zhuang_large(
    abca2_036, "domain_annotation", size=10, title="Zhuang-ABCA-2.036 Ground Truth"
)
plot_merfish_zhuang_large(
    abca2_036, "leiden", size=10, title="Zhuang-ABCA-2.036 NMI: 0.60, HOM: 0.57, COM: 0.62"
)